In [1]:
%%capture
!pip install torchsummary torchmetrics

# **Semanti Segmentation with SegNet**

* Emanuele Masiero (872695)
* Simone Vendramini (866229)
* Tommaso Ferrario (869005)

[Competition Link](https://www.kaggle.com/competitions/open-cv-tf-project-3-image-segmentation-round-3)


Dataset:
> M. Rahnemoonfar, T. Chowdhury, A. Sarkar, D. Varshney, M. Yari and R. R. Murphy, "FloodNet: A High Resolution Aerial Imagery Dataset for Post Flood Scene Understanding," in IEEE Access, vol. 9, pp. 89644-89654, 2021, doi: 10.1109/ACCESS.2021.3090981.


## **Imports and Utils**

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [13]:
import os
import sys
import torch
import time

# Add the parent directory to the path so we can import the module
sys.path.append("/content/drive/MyDrive/Colab Notebooks/Progetto AML/src")

from SegNet import SegNet
from Plots import plot_results
from torchsummary import summary
from Evaluation import Evaluation
from Utils import train, validate
from torch.utils.data import DataLoader
from FloodNetDataset import FloodNetDataset

In [4]:
batch_size = 16
num_workers = os.cpu_count()

baseline_path = "/content/drive/MyDrive/Colab Notebooks/Progetto AML/FloodNet-Supervised_v2.0_compressed"

specific_path = {
    "train": {"img": "train/train-org-img", "label": "ColorMasks-TrainSet"},
    "val": {"img": "val/val-org-img", "label": "ColorMasks-ValSet"},
    "test": {"img": "test/test-org-img", "label": "ColorMasks-TestSet"},
}

## **Dataset Loading**

In [5]:
train_data = FloodNetDataset(
    os.path.join(baseline_path, specific_path["train"]["label"]),
    os.path.join(baseline_path, specific_path["train"]["img"]),
)
train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True)

In [6]:
val_data = FloodNetDataset(
    os.path.join(baseline_path, specific_path["val"]["label"]),
    os.path.join(baseline_path, specific_path["val"]["img"]),
)

val_dataloader = DataLoader(val_data, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True)

In [7]:
test_data = FloodNetDataset(
    os.path.join(baseline_path, specific_path["test"]["label"]),
    os.path.join(baseline_path, specific_path["test"]["img"]),
)

test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=num_workers)

## **Model**

In [8]:
device = ("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

Using cuda device


In [9]:
model = SegNet(in_channels=3, out_channels=10).to(device)
model

SegNet(
  (enc1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (enc2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (enc3): Sequential(
    (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (dec3): Sequential(
    (0): Conv2d(256, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (dec2): Sequential(
    (0): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchN

In [10]:
summary(model, train_data[0][0].shape)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 512, 512]           1,792
       BatchNorm2d-2         [-1, 64, 512, 512]             128
              ReLU-3         [-1, 64, 512, 512]               0
            Conv2d-4        [-1, 128, 256, 256]          73,856
       BatchNorm2d-5        [-1, 128, 256, 256]             256
              ReLU-6        [-1, 128, 256, 256]               0
            Conv2d-7        [-1, 256, 128, 128]         295,168
       BatchNorm2d-8        [-1, 256, 128, 128]             512
              ReLU-9        [-1, 256, 128, 128]               0
           Conv2d-10        [-1, 128, 128, 128]         295,040
      BatchNorm2d-11        [-1, 128, 128, 128]             256
             ReLU-12        [-1, 128, 128, 128]               0
           Conv2d-13         [-1, 64, 256, 256]          73,792
      BatchNorm2d-14         [-1, 64, 2

In [11]:
epochs = 10
learning_rate = 1e-3

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
train_losses, val_losses = [], []
train_accuracies, val_accuracies = [], []

for epoch in range(epochs):
    train_loss, train_accuracy = train(
        train_dataloader, model, criterion, optimizer, device
    )
    val_loss, val_accuracy = validate(val_dataloader, model, criterion, device)

    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accuracies.append(train_accuracy)
    val_accuracies.append(val_accuracy)

    print(f"Epoch {epoch + 1}/{epochs}:")
    print(
        f"\tTrain Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy * 100:.2f}%"
    )
    print(f"\tVal Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy * 100:.2f}%")

Epoch 1/10:
	Train Loss: 1.1743, Train Accuracy: 61.31%
	Val Loss: 0.9649, Val Accuracy: 66.45%
Epoch 2/10:
	Train Loss: 0.9135, Train Accuracy: 68.93%
	Val Loss: 0.9880, Val Accuracy: 65.09%
Epoch 3/10:
	Train Loss: 0.8574, Train Accuracy: 70.92%
	Val Loss: 0.8409, Val Accuracy: 71.26%
Epoch 4/10:
	Train Loss: 0.8304, Train Accuracy: 71.86%
	Val Loss: 0.8057, Val Accuracy: 72.42%
Epoch 5/10:
	Train Loss: 0.8153, Train Accuracy: 72.22%
	Val Loss: 0.7998, Val Accuracy: 73.47%
Epoch 6/10:
	Train Loss: 0.8035, Train Accuracy: 72.61%
	Val Loss: 0.7901, Val Accuracy: 73.67%
Epoch 7/10:
	Train Loss: 0.7767, Train Accuracy: 73.38%
	Val Loss: 0.7665, Val Accuracy: 73.88%
Epoch 8/10:
	Train Loss: 0.7776, Train Accuracy: 73.90%
	Val Loss: 0.8551, Val Accuracy: 68.82%
Epoch 9/10:
	Train Loss: 0.7649, Train Accuracy: 73.89%
	Val Loss: 0.7435, Val Accuracy: 74.17%
Epoch 10/10:
	Train Loss: 0.7480, Train Accuracy: 74.61%
	Val Loss: 0.8163, Val Accuracy: 71.94%


In [ ]:
torch.save(model, 'model_segnet_ver2.pth')
torch.save(model.state_dict(), 'model_segnet_dict_ver2.pth')

## **Model Evaluation**

### Numerical Evaluation

In [ ]:
model = torch.load('/content/drive/MyDrive/Colab Notebooks/Progetto AML/Models/model_segnet_ver2.pth', map_location=torch.device(device))

<ipython-input-12-99ea977bb674>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load('/content/drive/MyDrive/Colab Notebooks/Progetto AML/Models/model_segnet_

In [ ]:
eval = Evaluation(model, test_dataloader, device)

In [ ]:
acc, dice, miou = eval.evaluate_model()
print(f"Accuracy: {acc * 100:.2f}%")
print(f"Dice Score: {dice * 100:.2f}%")
print(f"Mean IoU: {miou * 100:.2f}%")


Class occurrences in test set:
Class 0: 10166 images
Class 1: 15797 images
Class 2: 29456 images
Class 3: 19590 images
Class 4: 56739 images
Class 5: 45106 images
Class 6: 112628 images
Class 7: 10094 images
Class 8: 6566 images
Class 9: 199174 images
Accuracy: 71.65%
Dice Score: 73.18%
Mean IoU: 46.62%


### Visual Evaluation

In [ ]:
def plot_network_results(loader, model):
    image, label = next(iter(loader))
    image, label = image.to(device), label.to(device)

    model.eval()
    with torch.no_grad():
        pred = model(image)

    pred = pred.argmax(dim=1)
    # label = label.argmax(dim=3)
    for i in range(10):
        plot_results(image[i], pred[i].unsqueeze(0), label[i])

In [ ]:
plot_network_results(test_dataloader, model)

Output hidden; open in https://colab.research.google.com to view.